# How to use pyBarSim?
***
### Imports

In [ ]:
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from pybarsim import BarSim2D, BarSimPseudo3D

### Setup
Setting the backend for PyVista (3D visualization):

In [ ]:
pv.set_jupyter_backend('panel')

# BarSim in 2D

Define the initial elevation and the inflection points for the variations of sea level and sediment supply:

In [ ]:
initial_elevation = np.linspace(1000., 900., 200)
spacing = 100.

duration = 25000.
sea_level_curve = np.array([
    (0., 998.),
    (0.25*duration, 985.),
    (0.5*duration, 975.),
    (0.75*duration, 985.),
    (duration + 20., 998.)
])
sediment_supply_curve = np.array([
    (0., 25.),
    (0.25*duration, 25.),
    (0.5*duration, 25.),
    (0.75*duration, 5.),
    (duration + 20., 5.)
])

Initialize a *BarSim2D* object, run the simulation, and interpolate on a regular grid:

&#9888; This takes more time to run the first time because Numba needs to compile the Python code (around 15 s against less than 1 s for the following runs).

In [ ]:
%%time

barsim = BarSim2D(initial_elevation,
                  sea_level_curve,
                  sediment_supply_curve,
                  spacing=spacing,
                  ow_part=0.5,
                  mode=2,
                  W_fw=1.5,
                  W_event=6.,
                  Tidalampl=2,
                  Tide_Acc_VAR=100,
                  TidalSand=0.3,
                  dh_ratio_acc=200000,
                  TDcorr_SF=1.5,
                  TDcorr_OW=1.,
                  erodibility=0.1,
                  BB_max_width=500,
                  A_factor_BB=5,
                  A_factor_SF=10,
                  max_height_SF=1.,
                  max_height_BB=1.,
                  substrate_erosion=1,
                  fallout_rate_bb=0.,
                  fallout_rate_sf=0.0002,
                  texture=(5., 50., 125., 250.),
                  texture_ratio=(0.25, 0.25, 0.25, 0.25),
                  initial_substratum=(100., (0.2, 0.2, 0.3, 0.3)),
                  event=False,
                  preinterpolate_curves=True,
                  seed=42)
barsim.run(duration, dt_min=1., dt_fw=15.)
barsim.regrid(900., 1000., 0.5)

*run* creates *sequence_*, a [xarray](https://docs.xarray.dev/en/stable/) dataset containing the variations of sea level, sediment supply, elevation, stratigraphy, and facies through time:

In [ ]:
barsim.sequence_

*regrid* creates *record_* a [xarray](https://docs.xarray.dev/en/stable/) dataset containing the regions (0. air, 1. deposits, 2. substratum), the median grain size, and the stratigraphy interpolated in space:

In [ ]:
barsim.record_

Plot the resulting grid using [xarray's plotting functions](https://docs.xarray.dev/en/stable/user-guide/plotting.html?highlight=plotting):

In [ ]:
barsim.record_['Median grain size'].plot(figsize=(12, 4))

In [ ]:
barsim.record_['Major facies'].plot(figsize=(12, 4), cmap='Accent')

Animate the sea level and elevation through time using [matplotlib](https://matplotlib.org/stable/api/animation_api.html):

&#9888; This takes about 45 s to run.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

step = 8
time = barsim.sequence_['Time'][::step]
sea_level = barsim.sequence_['Sea level'][::step]
elevation = barsim.sequence_['Elevation'][::step]

def update(i):
    label_time.set_text(str(round(int(time[i]), -min(2, int(np.log10(time[i] + 1e-8))))) + ' yr')
    ax.collections.clear()
    ax.fill_between(barsim.sequence_['X'], elevation.min(), sea_level[i], edgecolor='#6baed6', facecolor='#c6dbef', zorder=0)
    ax.fill_between(barsim.sequence_['X'], elevation.min(), elevation[i], color='#fff7bc', zorder=1)
    k = 0
    for j in range(i):
        if j%15 == 0:
            elevation_prev = elevation[j].to_numpy().copy()
            elevation_prev[elevation_prev > elevation[i]] = elevation[i][elevation_prev > elevation[i]]
            lines_elevation_prev[k].set_data((barsim.sequence_['X'], elevation_prev))
            k += 1
    line_elevation.set_ydata(elevation[i])
    return label_time, line_elevation, lines_elevation_prev

ax.annotate('Time:', (0.85, 0.92), xycoords='axes fraction')
label_time = ax.annotate(str(round(int(time[0]), -min(2, int(np.log10(time[0] + 1e-8))))) + ' yr', (0.965, 0.92), ha='right', xycoords='axes fraction')
ax.fill_between(barsim.sequence_['X'], elevation.min(), sea_level[0], edgecolor='#6baed6', facecolor='#c6dbef', zorder=0)
ax.fill_between(barsim.sequence_['X'], elevation.min(), elevation[0], color='#fff7bc', zorder=1)
lines_elevation_prev = [ax.plot([], [], c='0.5', lw=0.5, zorder=2)[0] for i in range(len(elevation[::15]))]
line_elevation, = ax.plot(barsim.sequence_['X'], elevation[0], c='k', lw=1.5, zorder=3)
ax.set_xlim(barsim.sequence_['X'][0], barsim.sequence_['X'][-1])
ax.set_ylim(elevation.min(), elevation.max() + 20.)
ax.set_xlabel('x (m)')
ax.set_ylabel('z (m)')

plt.close()

ani = animation.FuncAnimation(fig, update, len(sea_level), interval=50)
HTML(ani.to_jshtml())

# BarSim in 2.5D

Define the initial elevation and the inflection points for the variations of sea level and sediment supply:

In [ ]:
initial_elevation = np.linspace(np.linspace(1000., 900., 200), np.linspace(1000., 850., 200), 200)
spacing = (100., 100.)

duration = 25000.
sea_level_curve = np.array([
    (0., 998.),
    (0.25*duration, 985.),
    (0.5*duration, 975.),
    (0.75*duration, 985.),
    (duration + 20., 998.)
])
sediment_supply_curve = np.array([
    np.tile([[0.], [0.25*duration], [0.5*duration], [0.75*duration], [duration + 20.]], 200),
    np.vstack([
        np.linspace(25., 5., 200),
        np.linspace(25., 5., 200),
        np.linspace(25., 5., 200),
        np.linspace(5., 1., 200),
        np.linspace(5., 1., 200),
    ])
]).T

Initialize a *BarSimPseudo3D* object, run the simulation and interpolate on a regular grid (in a single step):

In [ ]:
%%time

barsim = BarSimPseudo3D(initial_elevation,
                        sea_level_curve,
                        sediment_supply_curve,
                        spacing=spacing,
                        ow_part=0.5,
                        mode=2,
                        W_fw=1.5,
                        W_event=6.,
                        Tidalampl=2,
                        Tide_Acc_VAR=100,
                        TidalSand=0.3,
                        dh_ratio_acc=200000,
                        TDcorr_SF=1.5,
                        TDcorr_OW=1.,
                        erodibility=0.1,
                        BB_max_width=500,
                        A_factor_BB=5,
                        A_factor_SF=10,
                        max_height_SF=1.,
                        max_height_BB=1.,
                        substrate_erosion=1,
                        fallout_rate_bb=0.,
                        fallout_rate_sf=0.0002,
                        texture=(5., 50., 125., 250.),
                        texture_ratio=(0.25, 0.25, 0.25, 0.25),
                        initial_substratum=(200., (0.2, 0.2, 0.3, 0.3)),
                        event=False,
                        preinterpolate_curves=True,
                        seed=42)
barsim.run(850., 1000., 0.5, duration, dt_min=1., dt_fw=15.)

Here *run* directly creates *record_*:

In [ ]:
barsim.record_

Plot sections through the resulting 3D grid using [xarray's plotting functions](https://docs.xarray.dev/en/stable/user-guide/plotting.html?highlight=plotting):

In [ ]:
barsim.record_['Median grain size'][-10].plot(figsize=(8, 6))

In [ ]:
barsim.record_['Median grain size'][..., 100].plot(figsize=(12, 5))

Create a [PyVista](https://docs.pyvista.org/) mesh for 3D visualization of the sedimentary deposits:

In [ ]:
mesh = barsim.mesh(zscale=50.)

Visualize the full 2.5D deposits:

In [ ]:
p = pv.Plotter(notebook=True)
p.add_mesh(mesh.threshold(), scalars='Median grain size', lighting=False)
p.show()

Visualize slices through the deposits:

In [ ]:
p = pv.Plotter(notebook=True)
p.add_mesh(mesh.threshold().slice_along_axis(n=10, axis='x'), scalars='Median grain size', lighting=False)
p.add_mesh(mesh.threshold().slice_along_axis(n=10, axis='y'), scalars='Median grain size', lighting=False)
p.show()

In [ ]:
p = pv.Plotter(notebook=True)
p.add_mesh(mesh.threshold().slice_along_axis(n=10, axis='x'), scalars='Major facies', lighting=False)
p.add_mesh(mesh.threshold().slice_along_axis(n=10, axis='y'), scalars='Major facies', lighting=False)
p.show()